In [59]:
import pandas as pd

train_df = pd.read_csv('/Users/seok/Documents/Data Science/Kaggle/playground-series-s4e9/train.csv')
test_df = pd.read_csv('/Users/seok/Documents/Data Science/Kaggle/playground-series-s4e9/test.csv')

In [60]:
def fix_land_rover_model(df):
    # 'brand'가 'Land'이고 'model'이 'Rover'로 시작하는 경우
    land_rover_condition = (df['brand'] == 'Land') & (df['model'].str.startswith('Rover'))
    
    # 해당 조건을 만족하는 행의 'brand'를 'Land Rover'로 수정
    df.loc[land_rover_condition, 'brand'] = 'Land Rover'
    
    # 'model'에서 첫 번째 'Rover'만 제거하고 나머지 모델명을 유지
    df.loc[land_rover_condition, 'model'] = df.loc[land_rover_condition, 'model'].str.replace('^Rover', '', case=False, regex=True).str.strip()

    return df

# 데이터에 적용
train_df = fix_land_rover_model(train_df)


In [61]:
train_df['fuel_type'].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

In [62]:
# 결측치 처리
def clean_fuel_type(df):
    # NaN과 '-'를 함께 처리
    df['fuel_type'] = df['fuel_type'].fillna('not supported').replace(['-', '–'], 'not supported')
    return df

# engine 컬럼에서 fuel_type 추출
def extract_fuel_from_engine(df):
    extracted_fuel_types = []
    for engine in df['engine']:
        engine = engine.lower() if isinstance(engine, str) else ''
        
        if 'plug-in hybrid' in engine:
            extracted_fuel_types.append('Plug-In Hybrid')
        # 'hybrid' 관련 패턴을 좀 더 정확히 정의
        elif 'hybrid' in engine or 'gas/electric' in engine or 'gasoline/mild electric' in engine or 'electric/gas' in engine:
            extracted_fuel_types.append('Hybrid')
        # 'gasoline', 'gasoline fuel', 또는 'TFSI'가 포함된 경우 명확히 Gasoline으로 분류
        elif 'gasoline' in engine or 'gasoline fuel' in engine or 'TFSI' in engine:
            extracted_fuel_types.append('Gasoline')
        elif 'diesel' in engine:
            extracted_fuel_types.append('Diesel')
        elif 'flex fuel' in engine:
            extracted_fuel_types.append('E85 Flex Fuel')
        elif 'electric' in engine:
            extracted_fuel_types.append('Electric')
        else:
            extracted_fuel_types.append('not supported')
    
    df['fuel_type_extracted'] = extracted_fuel_types
    return df

def update_fuel_type(df):
    # 기존 fuel_type이 유효하지 않은 경우에만 업데이트
    df.loc[df['fuel_type'].isin(['not supported']) | df['fuel_type'].isna(), 'fuel_type'] = df['fuel_type_extracted']
    return df

def adjust_fuel_type_mismatches(df):
    df.loc[(df['fuel_type'] == 'Hybrid') & (df['fuel_type_extracted'] == 'Gasoline'), 'fuel_type_extracted'] = 'Hybrid'
    df['mismatch_flag'] = df.apply(lambda row: 'Significant Mismatch' if 
                                   (row['fuel_type'] == 'Gasoline' and row['fuel_type_extracted'] == 'Diesel') or
                                   (row['fuel_type'] == 'Diesel' and row['fuel_type_extracted'] == 'Gasoline') else '', axis=1)
    df.loc[df['fuel_type_extracted'] == 'not supported', 'fuel_type_extracted'] = df['fuel_type']
    return df

# Applying the logic
train_df = clean_fuel_type(train_df)
train_df = extract_fuel_from_engine(train_df)
train_df = update_fuel_type(train_df)
train_df = label_electric_brands(train_df)
train_df = adjust_fuel_type_mismatches(train_df)

# Check for significant mismatches
significant_mismatches = train_df[train_df['mismatch_flag'] != '']
print(significant_mismatches[['fuel_type', 'fuel_type_extracted', 'engine', 'mismatch_flag']].head())


     fuel_type fuel_type_extracted  \
41    Gasoline              Diesel   
1337  Gasoline              Diesel   
2228  Gasoline              Diesel   
2321  Gasoline              Diesel   
2350  Gasoline              Diesel   

                                          engine         mismatch_flag  
41    450.0HP 6.7L 8 Cylinder Engine Diesel Fuel  Significant Mismatch  
1337  161.0HP 2.0L 4 Cylinder Engine Diesel Fuel  Significant Mismatch  
2228  475.0HP 6.7L 8 Cylinder Engine Diesel Fuel  Significant Mismatch  
2321  475.0HP 6.7L 8 Cylinder Engine Diesel Fuel  Significant Mismatch  
2350  397.0HP 6.6L 8 Cylinder Engine Diesel Fuel  Significant Mismatch  


In [63]:
train_df['fuel_type'].value_counts()

fuel_type
Gasoline          165523
Hybrid              6833
Electric            5455
E85 Flex Fuel       5405
Diesel              3956
not supported        840
Plug-In Hybrid       521
Name: count, dtype: int64

In [48]:
def update_fuel_type_on_mismatch(df):
    # fuel_type과 fuel_type_extracted가 불일치하는 경우에만 fuel_type을 수정
    mismatch_condition = df['fuel_type'] != df['fuel_type_extracted']
    
    # fuel_type이 engine에서 추출한 정보와 다를 때만 수정
    df.loc[mismatch_condition, 'fuel_type'] = df.loc[mismatch_condition, 'fuel_type_extracted']
    
    return df

# Apply the mismatch update function
train_df = update_fuel_type_on_mismatch(train_df)

# 확인을 위해 fuel_type과 fuel_type_extracted가 달랐던 부분을 출력
mismatch_df = train_df[train_df['fuel_type'] != train_df['fuel_type_extracted']]

# 결과 출력
mismatch_df[['fuel_type', 'fuel_type_extracted', 'engine']].head()


,fuel_type,fuel_type_extracted,engine


In [49]:
train_df['fuel_type'].value_counts()

fuel_type
Gasoline          164833
Hybrid              7619
Electric            5605
E85 Flex Fuel       5435
Diesel              4188
not supported        840
Plug-In Hybrid        13
Name: count, dtype: int64

In [66]:
# Plug-In Hybrid -> Hybrid 변환
def convert_plug_in_hybrid_to_hybrid(df):
    df.loc[df['fuel_type'] == 'Plug-In Hybrid', 'fuel_type'] = 'Hybrid'
    return df

# Hydrogen 라벨링
def label_hydrogen_from_engine(df):
    hydrogen_conditions = df['engine'].str.contains('hydrogen|fuel cell', case=False, na=False)
    df.loc[hydrogen_conditions, 'fuel_type'] = 'Hydrogen'
    return df

# Electric 모델 수동 라벨링
def label_electric_models(df, electric_models):
    df.loc[df['model'].str.contains('|'.join(electric_models), case=False, na=False), 'fuel_type'] = 'Electric'
    return df

# Engine 정보에서 Electric 관련 패턴 라벨링
def label_electric_from_engine(df):
    electric_conditions = df['engine'].str.contains('battery|dual motor|motor', case=False, na=False)
    df.loc[electric_conditions, 'fuel_type'] = 'Electric'
    return df

# 'not supported' 처리
def handle_not_supported_fuel_types(df):
    df['fuel_type'] = df['fuel_type'].replace('–', 'not supported')
    df['fuel_type'] = df['fuel_type'].fillna('not supported')
    return df

# 최종 전처리 프로세스
def preprocess_fuel_types(df):
    df = convert_plug_in_hybrid_to_hybrid(df)
    df = label_hydrogen_from_engine(df)
    df = label_electric_models(df, electric_models=['mustang mach-e', 'taycan', 'bolt euv', 'kona ev', 'ioniq 5', 'hummer ev', 'EV6 Wind'])
    df = label_electric_from_engine(df)
    df = handle_not_supported_fuel_types(df)
    return df

# 데이터 전처리 적용
train_df = preprocess_fuel_types(train_df)

# 결과 확인
train_df['fuel_type'].value_counts()


fuel_type
Gasoline         165405
Hybrid             7307
Electric           5671
E85 Flex Fuel      5402
Diesel             3937
not supported       811
Name: count, dtype: int64

In [69]:
# Gasoline 강제 라벨링
def force_gasoline_update(df):
    # engine에 'gasoline fuel' 또는 'TFSI'가 포함된 경우 fuel_type을 무조건 Gasoline으로 설정
    gasoline_condition = df['engine'].str.contains('gasoline fuel|TFSI', case=False, na=False)
    
    # 해당 조건을 만족하는 경우 fuel_type을 'Gasoline'으로 강제 변경
    df.loc[gasoline_condition, 'fuel_type'] = 'Gasoline'
    
    return df

train_df = force_gasoline_update(train_df)

train_df['fuel_type'].value_counts()

fuel_type
Gasoline         166063
Hybrid             6665
Electric           5661
E85 Flex Fuel      5401
Diesel             3936
not supported       807
Name: count, dtype: int64

In [ ]:
train_df[train_df['fuel_type'] == 'not supported']

In [16]:
# Diesel-Gasoline mismatch cases: 0
# Empty DataFrame
# Columns: [fuel_type, engine, model, brand]
# Index: []

Diesel-Gasoline mismatch cases: 0
Empty DataFrame
Columns: [fuel_type, engine, model, brand]
Index: []


In [27]:
import pandas as pd

# Pandas 옵션을 설정하여 출력이 잘리지 않도록 설정
pd.set_option('display.max_columns', None)  # 모든 열을 출력
pd.set_option('display.max_rows', None)     # 모든 행을 출력
pd.set_option('display.max_colwidth', None) # 각 열의 값을 잘리지 않도록 설정

# # 'fuel_type'이 'Hybrid'인 데이터를 완전하게 출력
# train_df[train_df['fuel_type'] == 'Hybrid']



In [ ]:
train_df[train_df['fuel_type'] == 'Hybrid']

In [74]:
def force_label_electric_brands(df):
    # 전기차 브랜드 목록
    electric_only_brands = ['Tesla', 'Rivian', 'Lucid']
    
    # 해당 브랜드에 속하는 차량들의 fuel_type을 강제로 'Electric'으로 설정
    df.loc[df['brand'].isin(electric_only_brands), 'fuel_type'] = 'Electric'
    
    return df

# train_df에 적용
train_df = force_label_electric_brands(train_df)

In [75]:
# not supported 데이터 fuel_type 추론 (using ChatGPT)

# 도메인 지식을 바탕으로 연료 유형 추론 (더 많은 모델 포함)
fuel_type_map = {
    # Dodge
    "Dodge Challenger R/T Scat Pack": "Gasoline",
    "Dodge Challenger R/T": "Gasoline",
    "Dodge Ram 1500 SRT-10 Quad Cab": "Gasoline",
    "Dodge Charger R/T": "Gasoline",
    
    # Ford
    "Ford Mustang GT Premium": "Gasoline",
    "Ford Mustang EcoBoost Premium": "Gasoline",
    "Ford Bronco XLT": "Gasoline",
    "Ford F-150 Raptor": "Gasoline",
    "Ford F-150 XLT": "Gasoline",
    
    # Toyota
    "Toyota Land Cruiser Base": "Gasoline",
    "Toyota Prius Four": "Hybrid",
    "Toyota Corolla LE": "Gasoline",
    "Toyota Camry SE": "Gasoline",
    
    # Chevrolet
    "Chevrolet Protege DX": "Gasoline",
    "Chevrolet Corvette Base": "Gasoline",
    "Chevrolet Camaro SS": "Gasoline",
    "Chevrolet Avalanche 1500 LS": "Gasoline",
    
    # Tesla
    "Tesla Model S": "Electric",
    "Tesla Model 3": "Electric",
    "Tesla Model X": "Electric",
    "Tesla Model Y": "Electric",
    
    # Hyundai
    "Hyundai Ioniq 5": "Electric",
    "Hyundai Kona Electric": "Electric",
    "Hyundai Sonata SE": "Gasoline",
    
    # Honda
    "Honda Civic EX": "Gasoline",
    "Honda Accord Touring": "Gasoline",
    "Honda CR-V EX-L": "Gasoline",
    "Honda Insight EX": "Hybrid",
    
    # Nissan
    "Nissan Pathfinder SV": "Gasoline",
    "Nissan Leaf SV": "Electric",
    "Nissan Altima SL": "Gasoline",
    
    # BMW
    "BMW 330i": "Gasoline",
    "BMW X5 xDrive45e": "Plug-In Hybrid",
    "BMW i3": "Electric",
    
    # Mercedes-Benz
    "Mercedes-Benz C-Class": "Gasoline",
    "Mercedes-Benz GLE 350": "Gasoline",
    
    # Audi
    "Audi Q5 45 TFSI": "Gasoline",
    "Audi A3 Premium": "Gasoline",
    "Audi e-tron": "Electric",
    
    # Volkswagen
    "Volkswagen Jetta GLX VR6": "Gasoline",
    "Volkswagen ID.4": "Electric",
    
    # Kia
    "Kia EV6 Wind": "Electric",
    "Kia Soul EV": "Electric",
    "Kia Sorento LX": "Gasoline",
    
    # RAM
    "RAM 1500 Classic Warlock": "Gasoline",
    
    # Cadillac
    "Cadillac DeVille Base": "Gasoline",
    
    # Chrysler
    "Chrysler 300M Base": "Gasoline",
    "Chrysler Town & Country Touring-L": "Gasoline",
    
    # Mazda
    "Mazda Mazda6 i Grand Touring": "Gasoline",
    
    # 기타 추가적인 도메인 지식을 바탕으로 모델 및 연료 유형을 추가 가능
}

# 연료 유형 추론을 위해 train_df에 적용
def infer_fuel_type(df):
    # model을 기반으로 fuel_type을 추론하여 새로운 컬럼에 추가
    df['fuel_type_inferred'] = df['model'].map(fuel_type_map)
    
    # 추론되지 않은 모델들에 대해 'unknown' 처리
    df['fuel_type_inferred'].fillna('unknown', inplace=True)
    
    return df

# train_df에 적용
train_df = infer_fuel_type(train_df)

In [76]:
train_df['fuel_type'].value_counts()

fuel_type
Gasoline         166041
Hybrid             6665
Electric           5683
E85 Flex Fuel      5401
Diesel             3936
not supported       807
Name: count, dtype: int64

In [77]:
train_df.isnull().sum()

id                         0
brand                      0
model                      0
model_year                 0
milage                     0
fuel_type                  0
engine                     0
transmission               0
ext_col                    0
int_col                    0
accident                2452
clean_title            21419
price                      0
fuel_type_extracted        0
mismatch_flag              0
fuel_type_inferred         0
dtype: int64

In [79]:
# accident 결측치 처리: 'None reported'로 채우기
def handle_accident_missing(df):
    df['accident'].fillna('None reported', inplace=True)
    return df

# clean_title 결측치 처리: accident 정보를 기반으로 처리
def handle_clean_title_missing(df):
    df.loc[df['accident'] == 'None reported', 'clean_title'] = df.loc[df['accident'] == 'None reported', 'clean_title'].fillna('Yes')
    df['clean_title'].fillna('No', inplace=True)
    return df

# 결측치 처리 함수 적용
train_df = handle_accident_missing(train_df)
train_df = handle_clean_title_missing(train_df)

# 결과 확인
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   188533 non-null  int64 
 1   brand                188533 non-null  object
 2   model                188533 non-null  object
 3   model_year           188533 non-null  int64 
 4   milage               188533 non-null  int64 
 5   fuel_type            188533 non-null  object
 6   engine               188533 non-null  object
 7   transmission         188533 non-null  object
 8   ext_col              188533 non-null  object
 9   int_col              188533 non-null  object
 10  accident             188533 non-null  object
 11  clean_title          188533 non-null  object
 12  price                188533 non-null  int64 
 13  fuel_type_extracted  188533 non-null  object
 14  mismatch_flag        188533 non-null  object
 15  fuel_type_inferred   188533 non-nu